In [2]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 53.5 MB/s eta 0:00:00


# Worksheet 2: Solving problems using Z3

This week, you will keep encoding knowledge in a formal way and learn how Z3 can be used to answer our questions.

## New SMT2 syntax

To declare an FOL function, use
```
(declare-fun <Function name> (<Param. 1 type> <Param. 2 type> ...) <Return type>)
```
For example
```
(declare-fun square (Int Int) Int)
```
declares a function `square` with two `Int` parameters and `Int` return type.

Predicates in Z3 are functions that return `Bool`.


Z3 incorporates types such as `Bool` and `Int`.
In Exercise 1, however, it will be convenient to have a special type for blocks.
We will have exactly three blocks, so we can use an enumeration.
The syntax to define an enumeration in SMT2 is as follows:
```
(declare-datatypes () ((<Type name> <Value 1> <Value 2> <Value 3> ...)))
```
For example, you could define a data type `Animal` with possible values `dog`, `cat` and `fox`:
```
(declare-datatypes () ((Animal dog cat fox)))
```
Now `dog`, `cat` and `fox` are constants of type `Animal` referring to the corresponding values.

To complete this lab, you will also need to use quantifiers (keywords `forall` and `exists`).
For example, sentence $\forall x \forall y . P(x, y)$, where $x$ and $y$ are of type `MyType`, can be encoded in SMT2 as
```
(forall ((x MyType) (y MyType)) (P x y))
```
If you have only one quantifier, SMT2 still requires you to use all the brackets, e.g.:
```
(forall ((x MyType)) (Q x))
```


## Exercises

**Exercise 1.**

Recall the blocks example from Lecture 2 self-study exercise.  There were four sentences in the knowledge base:

1. $\mathit{OnTop}(a, b)$
2. $\mathit{OnTop}(b, c)$
3. $\mathit{Green}(a)$
4. $\lnot \mathit{Green}(c)$

The hypothesis was as follows:
$$
\exists x \exists y . \mathit{OnTop}(x, y) \land \mathit{Green}(x) \land \lnot \mathit{Green}(y).
$$

Encode the blocks example (both the knowledge base and the hypothesis) in Z3.
For now, use the SMT2 language.



In [ ]:
# Solution to exercise 1
smt2program = """
(declare-datatypes () ((Block a b c)))

(declare-fun Green (Block) Bool)
(declare-fun OnTop (Block Block) Bool)

(assert (OnTop a b))
(assert (OnTop b c))

(assert (Green a))
(assert (not (Green c)))

(assert (exists ((x Block) (y Block)) (and (and (OnTop x y) (Green x)) (not (Green y) ) ) ))
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

sat
[OnTop = [else -> True], Green = [a -> True, else -> False]]


**Exercise 2.**
Note that you just built a model of a piece of 'real world'; you mathematically described several objects and relations between them.  You also described what you believe to be true about this model&nbsp;– your _hypothesis_.

If you correctly encoded the knowledge base and the hypothesis, Z3 should tell you that your program is satisfiable (`sat`).  What does this mean?  Try to understand the model that Z3 found; for example, what is the interpretation of the `OnTop` predicate?

**Exercise 3.**
In fact, it would be nice to know the values of $x$ and $y$ that satisfy our hypothesis.  To obtain their values, remove the existential quantifier and declare $x$ and $y$ as constants.  Now, if you run the program again, you will also get the values of $x$ and $y$.  (Make sure you understand why this trick works!)

In [ ]:
# Solution to exercise 3
smt2program = """
(declare-datatypes () ((Block a b c)))

(declare-fun Green (Block) Bool)
(declare-fun OnTop (Block Block) Bool)

(declare-const x Block)
(declare-const y Block)

(assert (OnTop a b))
(assert (OnTop b c))

(assert (Green a))
(assert (not (Green c)))

(assert ( and (and (OnTop x y) (Green x)) (not (Green y) ) ) )
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

sat
[y = b,
 x = a,
 OnTop = [else -> True],
 Green = [a -> True, else -> False]]


**Exercise 4.**
There are two problems with the answer we get from Z3.
* Z3 does not prove that there is a green block on top of a non-green block no matter the colour of block $b$.  It simply assigns a colour to block $b$, and then the case becomes obvious &ndash; but only for this assignment.

* Moreover, Z3 suggests an interpretation for the predicate $\mathit{OnTop}$ which was not intended; in my case, it defined $\mathit{OnTop}(x, y)$ as TRUE for any inputs!
The cause of both of these issues is that we are asking if the program (the knowledge base and the hypothesis) is _satisfiable_, whereas in fact we want to know if the hypothesis is _entailed_ by the knowledge base.

**Z3 can only check satisfiability.  How can we use it to prove entailment?**
The trick is to negate our hypothesis and then ask if the program with the negated hypothesis is satisfiable.

* <ins>If the hypothesis is correct (is a logical consequence of our knowledge base)</ins> then every interepretation that satisfies the knowledge base also satisfies the hypothesis.  Then Z3 will not be able to find an interpretation that satisfies both the knowledge base and the negated hypothesis.  Hence it will return `unsat`.

* <ins>If the hypothesis is incorrect (is not a logical consequence of our knowledge base)</ins>, then there is at least one interpretation that satisfies the knowledge base but falsifies the hypothesis (obviously, satisfying the negated hypothesis); Z3 will find such an interpretation and return `sat`.

This principle can be summarised mathematically as follows:
* If $S \land \lnot \alpha$ is unsatisfiable then $S \models \alpha$.
* If $S \land \lnot \alpha$ is satisfiable then $S \not\models \alpha$.

Test that our hypothesis about the blocks is correct (i.e. entailed by the knowledge base) using the above trick.

In [ ]:
# Solution to exercise 4
smt2program = """
(declare-datatypes () ((Block a b c)))

(declare-fun Green (Block) Bool)
(declare-fun OnTop (Block Block) Bool)



(assert (OnTop a b))
(assert (OnTop b c))


(assert (Green a))
(assert (not (Green c)))


(assert (forall ((x Block) (y Block)) (not ( and (and (OnTop x y) (Green x)) (not (Green y) ) ) )))
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

unsat


**Exercise 5.**
Write a Python script (without the use of SMT2) that encodes your answer to Exercise 4.  It is easier to use integers in the Python implementation.
For example, you can assume that block $A$ is 1, block $B$ is 2 and block $C$ is 3.  Other than that, your implementation will be very similar to your answer to Exercise 4.

To declare a Z3 function from Python, use the `z3.Function` function.
The first parameter is the name of the function/predicate, followed by the types of the parameters and, finally, the return type.
For example, to declare a predicate $\mathit{OnTop}$ with two integer parameters, use the following syntax:
```
OnTop = Function('OnTop', IntSort(), IntSort(), BoolSort())
```

To use quantifiers, you will need functions `z3.Exists` and `z3.ForAll`, for example, the following code encodes $\forall x . \forall y . P(x, y)$ with the domain of discorse that includes all integer values:

In [ ]:
# Example of using quantifiers in Python
import z3

s = z3.Solver()

x = z3.Int('x')
y = z3.Int('y')
P = z3.Function('P', z3.IntSort(), z3.IntSort(), z3.BoolSort())
s.add(z3.ForAll([x, y], P(x, y)))

status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

sat
[P = [else -> True]]


In [ ]:
# Solution to exercise 5
smt2program = """
(declare-datatypes () ((Block a b c)))

(declare-fun Green (Block) Bool)
(declare-fun OnTop (Block Block) Bool)



(assert (OnTop a b))
(assert (OnTop b c))


(assert (Green a))
(assert (not (Green c)))


(assert (forall ((x Block) (y Block)) (not ( and (and (OnTop x y) (Green x)) (not (Green y) ) ) )))
"""
import z3

s = z3.Solver()

a = z3.Int('a')
b = z3.Int('b')
c = z3.Int('c')

OnTop = z3.Function('OnTop', z3.IntSort(), z3.IntSort(), z3.BoolSort())
Green = z3.Function('OnTop', z3.IntSort(), z3.BoolSort() )

s.add(OnTop (a, b))
s.add(OnTop (b, c))
s.add(Green (a))
s.add(z3.Not(Green (c)))

#  s.add(z3.ForAll([x,y], (z3.Not (z3.And((z3.And (OnTop (x,y), Green(x))), (z3.Not (Green (y))))))))
# s.add(z3.Exists([x,y], (z3.Not (z3.And((z3.And (OnTop (x,y), Green(x))), (z3.Not (Green (y))))))))

status = s.check()
print(status)
if status == z3.sat:
    print(s.model())


sat
[a = 0,
 c = 2,
 b = 1,
 OnTop = [else -> True],
 OnTop = [2 -> False, else -> True]]


**Exercise 6.**
Barber's paradox is a famous logical exercise by Bertrand Russell.
It is as follows:
* Anyone who does not shave himself must be shaved by the barber (note _the_ barber -- some unique person, not _a_ barber);
* Whomever the barber shaves, must not shave himself.

Encode this paradox in Z3 and check if it is satisfiable.

Hints:
* Introduce a predicate $\mathit{Shaves}(x,y)$ to indicate that $x$ shaves $y$.
* You will need a data type for people.
    You can define it using the recursive data type:
    ```
    (declare-datatypes () ((Person (first) (next (prev Person)))))
    ```
    Now `first` refers to the first person, `(next first)` refers to the second person, `(next (next first))` refers to the third person, etc.

* Define a constant `barber` of type `Person`.

In [4]:
# Solution to exercise 6
smt2program = """
(declare-datatypes () ((Person (first) (next (prev Person)))))

(declare-fun Shaves(Person Person) Bool)

(declare-const barber Person)

(assert (forall ((x Person)) (= (Shaves barber x) (not (Shaves barber barber)) )   ))


"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

unsat


Make sure you understand what the output of Z3 means; for example, does it mean that there is a solution to this paradox?

**Exercise 7.  (Expect this exercise to be much harder than any exercises before.)**
A disc jockey (DJ) composes a playlist of $n$ tracks.  The playlist consists of 'slow blocks', i.e. blocks of slow tracks, and 'fast blocks', i.e. blocks of fast tracks.  The fast and slow blocks alternate.  The DJ wants to follow several rules:

1. The slow block cannot include more than two tracks.
2. The fast block has to include at least three tracks.
3. Tracks 1, 8, 15, 22, ... have to be slow.  (Assume indexing from 1.)
4. Tracks 6, 11, 16, 21, ... have to be fast.

Write a Python script that finds the desired structure of the playlist using Z3 for a given $n$.  Produce two implementations: one using $n$ propositional variables, and another one using one predicate with a parameter.

Test each approach with $n = 30$ and $n = 31$.  Verify that the produced solutions are correct.

Hints:
* The key to this exercise is deciding on the meaning of those predicates.  Once you fix the meaning of them (it really helps to write it down!), you can use conditions such as 'if the $i$th track is slow and the $(i+1)$th track is slow then...'.
* It helps a lot to write down such conditions on a piece of paper before encoding them in Python.
* A condition can be encoded using implication.
* You will create such conditions for each $i$.
* In your solution that uses multiple predicate variables, you will need to create a list of Boolean constants and then use loops to encode the rules and print the solution.

In [ ]:
# Solution to exercise 7 with n propositional variables
import z3

s = z3.Solver()
n = 31
S = [z3.Bool(f'S_{i+1}') for i in range(n)]

for i in range(1, n - 1):
        s.add(
            z3.Not(z3.And(S[i-1], S[i], S[i+1]))
        )

for i in range(1, n - 1):
        s.add(
            z3.Not(
                z3.And(S[i-1], z3.Not(S[i]), S[i+1])
            )
        )

for i in range(1, n - 2):
        s.add(
            z3.Implies(
                z3.And(S[i-1], z3.Not(S[i]), z3.Not(S[i+1])),
                z3.Not(S[i+2])
            )
        )


for i in range(1, n + 1):
        if (i - 1) % 7 == 0:
            s.add(S[i-1] == True)

for i in range(6, n + 1):
        if (i - 6) % 5 == 0:
            s.add(S[i-1] == False)




status = s.check()
print(status)
if status == z3.sat:
    print(s.model())


sat
[S_15 = True,
 S_26 = False,
 S_14 = False,
 S_30 = False,
 S_16 = False,
 S_17 = False,
 S_19 = False,
 S_23 = False,
 S_8 = True,
 S_5 = False,
 S_2 = False,
 S_9 = False,
 S_1 = True,
 S_25 = False,
 S_13 = False,
 S_7 = False,
 S_20 = False,
 S_12 = False,
 S_4 = False,
 S_22 = True,
 S_29 = True,
 S_6 = False,
 S_11 = False,
 S_27 = False,
 S_10 = False,
 S_24 = False,
 S_3 = False,
 S_31 = False,
 S_28 = False,
 S_21 = False,
 S_18 = False]


In [ ]:
# Solution to exercise 7 with a single parameterised predicate
import z3
s = z3.Solver()

n=30

Check = z3.Function('Check', z3.IntSort(), z3.BoolSort())

i = z3.Int('i')

for k in range(1, n+1 , 7):
    s.add(Check(k))

for k in range(6, n+1 , 5):
    s.add(z3.Not(Check(k)))


s.add(z3.And(z3.And(i<=n,i>=1),z3.Not(z3.And(z3.And(Check(i),Check(i+1)),Check(i+2)))))
s.add(z3.And(z3.And(i<=n,i>=1),z3.Not(z3.And(z3.And(z3.Not(Check(i)),z3.Not(Check(i+1))),Check(i+2)))))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())



sat
[i = 9,
 Check = [1 -> True,
          8 -> True,
          15 -> True,
          22 -> True,
          29 -> True,
          else -> False]]
